In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold
import lightgbm as lgb

In [28]:
train_df=pd.read_csv(r'C:\Users\weiping\Desktop\nlp\train.csv',encoding='gbk')
test_df=pd.read_csv(r'C:\Users\weiping\Desktop\nlp\test.csv',encoding='gbk')

In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5070 entries, 0 to 5069
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   编号        5070 non-null   int64  
 1   性别        5070 non-null   int64  
 2   出生年份      5070 non-null   int64  
 3   体重指数      5070 non-null   float64
 4   糖尿病家族史    5070 non-null   object 
 5   舒张压       4823 non-null   float64
 6   口服耐糖量测试   5070 non-null   float64
 7   胰岛素释放实验   5070 non-null   float64
 8   肱三头肌皮褶厚度  5070 non-null   float64
 9   患有糖尿病标识   5070 non-null   int64  
dtypes: float64(5), int64(4), object(1)
memory usage: 396.2+ KB


In [30]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   编号        1000 non-null   int64  
 1   性别        1000 non-null   int64  
 2   出生年份      1000 non-null   int64  
 3   体重指数      1000 non-null   float64
 4   糖尿病家族史    1000 non-null   object 
 5   舒张压       951 non-null    float64
 6   口服耐糖量测试   1000 non-null   float64
 7   胰岛素释放实验   1000 non-null   float64
 8   肱三头肌皮褶厚度  1000 non-null   float64
dtypes: float64(5), int64(3), object(1)
memory usage: 70.4+ KB


In [31]:
print(train_df.shape, test_df.shape)
print(train_df.dtypes, test_df.dtypes)

(5070, 10) (1000, 9)
编号            int64
性别            int64
出生年份          int64
体重指数        float64
糖尿病家族史       object
舒张压         float64
口服耐糖量测试     float64
胰岛素释放实验     float64
肱三头肌皮褶厚度    float64
患有糖尿病标识       int64
dtype: object 编号            int64
性别            int64
出生年份          int64
体重指数        float64
糖尿病家族史       object
舒张压         float64
口服耐糖量测试     float64
胰岛素释放实验     float64
肱三头肌皮褶厚度    float64
dtype: object


In [32]:
train_df.isnull().mean(axis=0)

编号          0.000000
性别          0.000000
出生年份        0.000000
体重指数        0.000000
糖尿病家族史      0.000000
舒张压         0.048718
口服耐糖量测试     0.000000
胰岛素释放实验     0.000000
肱三头肌皮褶厚度    0.000000
患有糖尿病标识     0.000000
dtype: float64

In [33]:
test_df.isnull().mean(axis=0)

编号          0.000
性别          0.000
出生年份        0.000
体重指数        0.000
糖尿病家族史      0.000
舒张压         0.049
口服耐糖量测试     0.000
胰岛素释放实验     0.000
肱三头肌皮褶厚度    0.000
dtype: float64

In [34]:
train_df.corr()

,编号,性别,出生年份,体重指数,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,患有糖尿病标识
编号,1.000000,0.006603,-0.006693,0.000028,0.003495,-0.005840,0.020441,0.030330,0.027435
性别,0.006603,1.000000,-0.119563,0.075186,0.078870,0.011463,-0.053597,0.014037,0.031480
出生年份,-0.006693,-0.119563,1.000000,-0.074603,-0.154631,0.002085,0.058585,-0.013111,-0.068225
体重指数,0.000028,0.075186,-0.074603,1.000000,0.159903,-0.001796,-0.034507,0.026321,0.377919
舒张压,0.003495,0.078870,-0.154631,0.159903,1.000000,-0.020317,-0.206663,0.076147,0.157421
口服耐糖量测试,-0.005840,0.011463,0.002085,-0.001796,-0.020317,1.000000,0.093715,-0.006483,0.178133
胰岛素释放实验,0.020441,-0.053597,0.058585,-0.034507,-0.206663,0.093715,1.000000,-0.015479,0.156656
肱三头肌皮褶厚度,0.030330,0.014037,-0.013111,0.026321,0.076147,-0.006483,-0.015479,1.000000,0.410667
患有糖尿病标识,0.027435,0.031480,-0.068225,0.377919,0.157421,0.178133,0.156656,0.410667,1.000000


In [35]:
train_df['糖尿病家族史'].value_counts()

无记录               2897
叔叔或姑姑有一方患有糖尿病     1084
父母有一方患有糖尿病         875
叔叔或者姑姑有一方患有糖尿病     214
Name: 糖尿病家族史, dtype: int64

In [36]:
dict_糖尿病家族史 = {'无记录': 0,'叔叔或姑姑有一方患有糖尿病': 1,'叔叔或者姑姑有一方患有糖尿病': 1,'父母有一方患有糖尿病': 2}

In [37]:
train_df['糖尿病家族史'] = train_df['糖尿病家族史'].map(dict_糖尿病家族史)
test_df['糖尿病家族史'] = test_df['糖尿病家族史'].map(dict_糖尿病家族史)
train_df['糖尿病家族史'] = train_df['糖尿病家族史'].astype('category')
test_df['糖尿病家族史'] = train_df['糖尿病家族史'].astype('category')

In [38]:
train_df['体重指数_round'] = train_df['体重指数'] // 10
test_df['体重指数_round'] = train_df['体重指数'] // 10

In [39]:
train_df['口服耐糖量测试'] = train_df['口服耐糖量测试'].replace(-1, np.nan)
test_df['口服耐糖量测试'] = test_df['口服耐糖量测试'].replace(-1, np.nan)

In [40]:
train_df.groupby('糖尿病家族史')['口服耐糖量测试'].mean()

糖尿病家族史
0    5.896475
1    5.967212
2    6.125952
Name: 口服耐糖量测试, dtype: float64

In [41]:
train_df['口服耐糖量测试_diff'] = train_df['口服耐糖量测试'] - train_df.groupby('糖尿病家族史').transform(lambda x: x-x.mean())['口服耐糖量测试']
test_df['口服耐糖量测试_diff'] = test_df['口服耐糖量测试'] - test_df.groupby('糖尿病家族史').transform(lambda x: x-x.mean())['口服耐糖量测试']

In [42]:
train_df['舒张压'].mean()

89.42359527265188

In [43]:
test_df['舒张压'].mean()

89.63827549947423

In [44]:
train_df['舒张压'].fillna(89, inplace=True)
test_df['舒张压'].fillna(89, inplace=True)

In [45]:
train_df['性别'] = train_df['性别'].astype('category')
test_df['性别'] = train_df['性别'].astype('category')

In [46]:
train_df['年龄'] = 2022 - train_df['出生年份']
test_df['年龄'] = 2022 - test_df['出生年份']

In [47]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5070 entries, 0 to 5069
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   编号            5070 non-null   int64   
 1   性别            5070 non-null   category
 2   出生年份          5070 non-null   int64   
 3   体重指数          5070 non-null   float64 
 4   糖尿病家族史        5070 non-null   category
 5   舒张压           5070 non-null   float64 
 6   口服耐糖量测试       4821 non-null   float64 
 7   胰岛素释放实验       5070 non-null   float64 
 8   肱三头肌皮褶厚度      5070 non-null   float64 
 9   患有糖尿病标识       5070 non-null   int64   
 10  体重指数_round    5070 non-null   float64 
 11  口服耐糖量测试_diff  4821 non-null   float64 
 12  年龄            5070 non-null   int64   
dtypes: category(2), float64(7), int64(4)
memory usage: 446.0 KB


In [48]:
train_df=train_df.dropna(axis=0,how='any')
test_df=test_df.dropna(axis=0,how='any')

In [49]:
train_df.isnull().mean(axis=0)

编号              0.0
性别              0.0
出生年份            0.0
体重指数            0.0
糖尿病家族史          0.0
舒张压             0.0
口服耐糖量测试         0.0
胰岛素释放实验         0.0
肱三头肌皮褶厚度        0.0
患有糖尿病标识         0.0
体重指数_round      0.0
口服耐糖量测试_diff    0.0
年龄              0.0
dtype: float64

In [50]:
test_df.isnull().mean(axis=0)

编号              0.0
性别              0.0
出生年份            0.0
体重指数            0.0
糖尿病家族史          0.0
舒张压             0.0
口服耐糖量测试         0.0
胰岛素释放实验         0.0
肱三头肌皮褶厚度        0.0
体重指数_round      0.0
口服耐糖量测试_diff    0.0
年龄              0.0
dtype: float64

## Linear regression

model = make_pipeline(
    MinMaxScaler(),
    DecisionTreeClassifier()
)
model.fit(train_df.drop(['编号', '患有糖尿病标识'], axis=1), train_df['患有糖尿病标识'])

test_df['label'] = model.predict(test_df.drop(['编号'], axis=1))
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit_linear.csv', index=None)

##  Random forest model 

rf=RandomForestRegressor(n_estimators=500,n_jobs=-1)
rf.fit(train_df.drop(['编号', '患有糖尿病标识'], axis=1), train_df['患有糖尿病标识'])
test_df['label']=rf.predict(test_df.drop(['编号'], axis=1))
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit_rf.csv', index=None)

## lgb model

In [51]:
def run_model_cv(model, kf, X_tr, y, X_te, cate_col=None):
    train_pred = np.zeros( (len(X_tr), len(np.unique(y))) )
    test_pred = np.zeros( (len(X_te), len(np.unique(y))) )

    cv_clf = []
    for tr_idx, val_idx in kf.split(X_tr, y):
        x_tr = X_tr.iloc[tr_idx]; y_tr = y.iloc[tr_idx]

        x_val = X_tr.iloc[val_idx]; y_val = y.iloc[val_idx]

        call_back = [
            lgb.early_stopping(50),
        ]
        eval_set = [(x_val, y_val)]
        model.fit(x_tr, y_tr, eval_set=eval_set, callbacks=call_back, verbose=-1)

        cv_clf.append(model)

        train_pred[val_idx] = model.predict_proba(x_val)
        test_pred += model.predict_proba(X_te)

    test_pred /= kf.n_splits
    return train_pred, test_pred, cv_clf

clf = lgb.LGBMClassifier(
    max_depth=3, 
    n_estimators=4000, 
    n_jobs=-1, 
    verbose=-1,
    verbosity=-1,
    learning_rate=0.1,
)

train_pred, test_pred, cv_clf = run_model_cv(
    clf, KFold(n_splits=5),
    train_df.drop(['编号', '患有糖尿病标识'], axis=1),
    train_df['患有糖尿病标识'],
    test_df.drop(['编号'], axis=1),
)

print((train_pred.argmax(1) == train_df['患有糖尿病标识']).mean())
test_df['label'] = test_pred.argmax(1)
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit_lgb.csv', index=None)

E:\anaconda\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
E:\anaconda\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
E:\anaconda\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[242]	valid_0's binary_logloss: 0.133147
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
Training until validation scores don't improve for 50 rounds


E:\anaconda\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
E:\anaconda\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
E:\anaconda\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Early stopping, best iteration is:
[190]	valid_0's binary_logloss: 0.122968
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
Training until validation scores don't improve for 50 rounds


E:\anaconda\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
E:\anaconda\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
E:\anaconda\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Early stopping, best iteration is:
[226]	valid_0's binary_logloss: 0.115888
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
Training until validation scores don't improve for 50 rounds


E:\anaconda\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
E:\anaconda\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
E:\anaconda\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Early stopping, best iteration is:
[265]	valid_0's binary_logloss: 0.130943
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
Training until validation scores don't improve for 50 rounds


E:\anaconda\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
E:\anaconda\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
E:\anaconda\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Early stopping, best iteration is:
[179]	valid_0's binary_logloss: 0.108291
0.9572702758763741
